## Pulls day-ahead market data from Entsoe transparency platform

In [ ]:
# import urllib.request

# #Switzerland 60min (2016-2024)
# pref = "https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/export?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=16.09."
# suff = "+00%3A00%7CCET%7CDAY&biddingZone.values=CTY%7C10YCH-SWISSGRIDZ!BZN%7C10YCH-SWISSGRIDZ&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC%2B1)+%2F+CEST+(UTC%2B2)&dataItem=ALL&timeRange=YEAR&exportType=CSV"

# for i in range(2016, 2025):
#     print(pref + str(i) + suff)

# #Germany (DE-AT-LU) 60min (2015-2018)
# pref = "https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/export?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=01.09."
# suff = "+00%3A00%7CCET%7CDAY&biddingZone.values=CTY%7C10Y1001A1001A83F!BZN%7C10Y1001A1001A63L&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC%2B1)+%2F+CEST+(UTC%2B2)&dataItem=ALL&timeRange=YEAR&exportType=CSV"

# for i in range(2015, 2019):
#     print(pref + str(i) + suff)

# #Germany (DE-LU) 60min (2018-2024)
# pref = "https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/export?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=16.09."
# suff = "+00%3A00%7CCET%7CDAY&biddingZone.values=CTY%7C10Y1001A1001A83F!BZN%7C10Y1001A1001A82H&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC%2B1)+%2F+CEST+(UTC%2B2)&dataItem=ALL&timeRange=YEAR&exportType=CSV"

# for i in range(2018, 2025):
#     print(pref + str(i) + suff)

# #Spain 60min (2015-2024)

# pref = "https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/export?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=16.09."
# suff = "+00%3A00%7CCET%7CDAY&biddingZone.values=CTY%7C10YES-REE------0!BZN%7C10YES-REE------0&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC%2B1)+%2F+CEST+(UTC%2B2)&dataItem=ALL&timeRange=YEAR&exportType=CSV"

# for i in range(2015, 2025):
#     print(pref + str(i) + suff)

# #Italy
# #Spain 60min (2015-2024)

# pref = "https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/export?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=16.09."
# suff = "+00%3A00%7CCET%7CDAY&biddingZone.values=CTY%7C10YIT-GRTN-----B!BZN%7C10Y1001A1001A71M&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC%2B1)+%2F+CEST+(UTC%2B2)&dataItem=ALL&timeRange=YEAR&exportType=CSV"

# for i in range(2015, 2025):
#     print(pref + str(i) + suff)

## Check data range for day-ahead ENTSO-e data

In [2]:
import pandas as pd
pd.set_option("future.no_silent_downcasting", True)
import numpy as np
import os

In [3]:
# Main ENTSO-e data folder
data_folder = '../../20_data/entsoe_data/DE'

# Subfolders
DA_15min_DE_LU = '/DA_15min/DE-LU/'#Day-ahead Prices_' 
DA_60min_DE_LU = '/DA_60min/DE-LU/'#Day-ahead Prices_'
DA_60min_DE_AT_LU = '/DA_60min/DE-AT-LU/'#Day-ahead Prices_'

In [5]:
def date_range(product, offset):
    series_DA_aggregated = pd.Series()
    se = pd.Series(os.listdir(data_folder+product))
    start_years = se[se.str.contains('Day-ahead Prices_')].str.slice(17,21).sort_values() # Get files that follow pattern and sort by year
    for i in start_years:
        df_DA = pd.read_csv(data_folder+product+ f'Day-ahead Prices_{i}01010000-{str(int(i)+1)}01010000.csv') # Get year file
        # Drop completely empty columns and columns 'Currency' and set index to time
        df_DA = df_DA.drop(columns=['Currency']).dropna(axis=1, how='all').set_index('MTU (CET/CEST)')
        df_DA.index = pd.to_datetime(df_DA.index.str.slice(0,16), format="%d.%m.%Y %H:%M")
        if len(df_DA.columns) != 0: #If there's still a columns
            # Convert all non numbers and dots to nans and dropna
            df_DA[df_DA.columns[0]] = df_DA[df_DA.columns[0]].replace('^[^0-9.]+$', np.nan, regex=True).astype(float)
            df_DA.dropna(inplace=True)
            series_DA = df_DA.squeeze() # Convert to series
            if not(series_DA_aggregated.empty): #If there are values in aggregated, add current series
                series_DA_aggregated = pd.concat([series_DA_aggregated,series_DA])
            else: #Else define as the current DA timeseries
                series_DA_aggregated = series_DA
    # Offset for plotting multiple
    # (series_DA_aggregated + offset).plot(title = '')
    # print(min(series_DA_aggregated.index))
    # print(max(series_DA_aggregated.index))
    return series_DA_aggregated

In [7]:
# series_DA_aggregated = date_range(DA_60min_DE_AT_LU, 0)

# series_DA_aggregated = date_range(DA_60min_DE_LU, 0)

# series_DA_aggregated = date_range(DA_15min_DE_LU, 0)

# Check length of index vs timerange to see missing values
print(len(series_DA_aggregated))
print(((max(series_DA_aggregated.index)-min(series_DA_aggregated.index))/pd.Timedelta('1h')))
print(((max(series_DA_aggregated.index)-min(series_DA_aggregated.index))/pd.Timedelta('0.25h')))


210716
52703.75
210815.0
